In [572]:
import os
import sys
from glob import glob
from tqdm import tqdm
import math
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
import cv2

import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader
import albumentations as A
sys.path.insert(0, '../')
from utils import load_json, save_pickle
from model import VanillaEfficientNet, load_model
from dataset import TrainDataset, EvalDataset
from torchvision import transforms
from transform_settings import configure_transform
from albumentations import CenterCrop, Blur, Cutout, Equalize, GaussianBlur, GaussNoise, GlassBlur, GridDistortion, Lambda, MedianBlur, MotionBlur, Normalize, RandomBrightnessContrast, RandomFog, Solarize, Resize, ToGray
from albumentations.pytorch.transforms import ToTensor

In [193]:
transform = transforms.Compose([
    transforms.Resize((512, 384), Image.BILINEAR),
    transforms.CenterCrop((384, 384)),
    # transforms.RandomResizedCrop((224, 224)),
    ])

data_config = {
    'root': '../preprocessed_stratified/train', 
    'transform': None, 
    'task': 'main',
    'meta_path': '../preprocessed_stratified/metadata.json'
    }
dataset = TrainDataset(**data_config)
model = VanillaEfficientNet(n_class=18)
loader = DataLoader(dataset, batch_size=4)

Loaded pretrained weights for efficientnet-b3


In [697]:
def get_similarity(pixel_dist_left, pixel_dist_right):
    """
    픽셀 분포 간 유사도를 측정하는 함수
    분포 간 intersection을 유사도로 가정하여 높을 수록 유사도가 높다
    """
    hist_left = cv2.calcHist(images=[pixel_dist_left], channels=[0], mask=None, histSize=[32], ranges=[0,256])
    hist_right = cv2.calcHist(images=[pixel_dist_right], channels=[0], mask=None, histSize=[32], ranges=[0,256])
    similarity = cv2.compareHist(H1=hist_left, H2=hist_right, method=cv2.HISTCMP_INTERSECT)
    return similarity

def split_image(img: np.array, v, scale):
    """수직선을 기준으로 이미지를 분할
    좌우로 이미지를 분할했을 때, 가로 길이가 같도록 가로 길이를 조정
    
    Args
    ---
    img: np.array, grayscale
    v: 수직선의 위치
    scale: crop 과정에 활용된 scale
    """
    if v >= scale:
        margin = scale - v%scale
        pixel_dist_left = img[:, v-margin:v]
        pixel_dist_right = img[:, v:]
    else:
        margin = v%scale
        pixel_dist_left = img[:, :v]
        pixel_dist_right = img[:, v:v+margin]
    return pixel_dist_left, pixel_dist_right


def get_vline(img_resized: np.array, scale: int=64):
    v_list = [i+int(scale*(2/5)) for i in range(scale//5)][::2]
    margin_list = [i+int(scale*(2/5)) for i in range(scale//5)][::2]
    importances = np.zeros(len(v_list))

    for idx, v in enumerate(v_list):
        pixel_dist_left, pixel_dist_right = split_image(img_resized, v, scale)
        similarity = get_similarity(pixel_dist_left, pixel_dist_right)
        importances[idx] += (similarity / (v*scale)) # normalize
    v = v_list[np.argsort(importances)[3]] # (경험적) importance가 가장 높은 값이 아닌 2~3번째로 높은 값이 원하는 수직선이 나오는 경우가 많음
    return v


def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w


def get_hline(img: np.array, v):
    UPPER_QUANTILE = 0.35
    LOWER_QUANTILE = 1 - UPPER_QUANTILE
    FACTOR = 8

    v0 = moving_average(img_resized[:, v], FACTOR) # 단순이동평균
    v1 = np.roll(v0, shift=1, axis=0)
    sma_interval = v0.shape[0]
    offset = int(sma_interval * LOWER_QUANTILE)

    h_upper = np.argmax(np.abs(v0[1:int(sma_interval*UPPER_QUANTILE)] - v1[1:int(sma_interval*UPPER_QUANTILE)])) + FACTOR - 1
    h_lower = np.argmax(np.abs(v0[int(sma_interval*LOWER_QUANTILE):-1] - v1[int(sma_interval*LOWER_QUANTILE):-1])) + offset - FACTOR + 1

    return h_upper, h_lower


def crop(img, scale: int=64, p=0.5, rescaled=True):
    if np.random.uniform(0, 1) >= p: # 시행하지 않을 경우
        return img
        
    transform = transforms.Compose([transforms.Resize((int(scale*(4/3)), scale)), transforms.Grayscale()])
    # transform = A.Compose([Resize((int(scale*(4/3)), scale)), ToGray])
    img_resized = np.array(transform(img))
    MAX_H, MAX_W = np.array(img).shape[:2]

    vline = get_vline(img_resized, scale)
    h_upper, h_lower = get_hline(img_resized, vline)

    if rescaled: # 입력할 때의 이미지 크기에 맞게 수직선의 위치를 조정
        origin_v_scale = np.array(img).shape[1]
        origin_h_scale = np.array(img).shape[0]

        vline = int(origin_v_scale * (vline / scale))
        h_lower = int(origin_h_scale * (h_lower / int(scale*(4/3))))
        h_upper = int(origin_h_scale * (h_upper / int(scale*(4/3))))

    hline = int((h_lower + h_upper) / 2)
    height = abs(h_upper - h_lower)
    height = int(1.1 * height)
    width = int(height * (3/4))

    coord_y = np.clip([hline-height, hline+height], 0, MAX_H)
    coord_x = np.clip([vline-width, vline+width], 0, MAX_W)
    cropped = np.array(img)[coord_y[0]:coord_y[1], coord_x[0]:coord_x[1], :]

    return Image.fromarray(cropped)

In [743]:
268 * (4/3)

357.3333333333333

In [751]:
234 * (4/3)

312.0

In [763]:
transform = transforms.Compose([
    transforms.Lambda(lambda x: crop(x)),
    transforms.Resize((312, 234)),
    transforms.RandomCrop((224, 224)),
    transforms.ToTensor()
    ])

def foo():
    transform(dataset[20][0])
%timeit foo()

6.88 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [713]:
transform(image=np.array(img))

TypeError: __call__() got an unexpected keyword argument 'image'

In [577]:
aug(image=np.array(sample))

TypeError: <lambda>() got an unexpected keyword argument 'cols'